# Rayyan SDK for Python

## Installation

After installing the package you can import it and create a `rayyan` instance as follows:

> Note: the python package is not yet available on PyPI, so you need to install it from the source code.

In [2]:
from pprint import pprint
from rayyan import Rayyan

rayyan = Rayyan("creds.json")

The `creds.json` file should contain the following:

```json
{
    "access_token": "...",
    "refresh_token": "..."
}
```

A valid file can be downloaded by signing in to Rayyan then going to My Account.

> Note: this is not supported yet in production for all users.

## Accessing signed-in user data

The simplest SDK call is the one that retrieves the signed-in user's data. This is done by calling the `get_info` method of `User` object:

In [3]:
from rayyan.user import User

user = User(rayyan).get_info()

# Remove request_token before printing
user.pop('request_token', None)
# Remove some other fields
user.pop('subscription', None)
# user.pop('feature_flags', None)
pprint(user)

{'displayName': 'MR test test2',
 'feature_flags': {'add_review_metadata_to_review_form': True,
                   'auto_resolver': True,
                   'customize_by_filter': True,
                   'feos_access': True,
                   'feos_banner': True,
                   'full_text': True,
                   'live_chat': True,
                   'mendeley_import': True,
                   'mobile_access': False,
                   'phone_support': False,
                   'pico_extraction': True,
                   'prisma': True,
                   'revoked_roles': True,
                   'training': True},
 'firstName': 'test',
 'lastName': 'test2',
 'publicName': 'test test2',
 'userId': 266,
 'user_email': 'mohammad+team+owner@rayyan.ai'}


## Retrieving the list of all reviews

In [4]:
from rayyan.review import Review
rayyan_review = Review(rayyan)
reviews = rayyan_review.get_all()
pprint(reviews)
print(f'Found {len(reviews["owned_reviews"])} owned reviews and {len(reviews["collab_reviews"])} shared reviews.')

{'collab_reviews': [],
 'collab_reviews_total': 0,
 'owned_reviews': [{'accessed_at': '2023-09-07T13:21:35.428Z',
                    'articles_etag': '3408:10',
                    'created_at': '2023-09-06T13:08:40.060Z',
                    'inclusions_etag': '2023-09-07 11:16:41 UTC',
                    'is_blind': True,
                    'labels_etag': '2023-09-07 13:21:32 UTC',
                    'maybe_decision_supported': True,
                    'owner': {'abbrev': 'test',
                              'email': 'mohammad+team+owner@rayyan.ai',
                              'enterprise_power_user?': False,
                              'global_owner_user?': False,
                              'guest?': False,
                              'has_fulltext_access': True,
                              'id': 266,
                              'most_recent_active_single_team_subscription': None,
                              'most_recent_active_team_subscription': {'cancelled_at

## Working on the first owned review

### Printing some review key information

In [5]:
my_review = reviews['owned_reviews'][0]
# pprint(my_review)
print(f"Here is the first review in the list of owned reviews: '{my_review['title']}' created on '{my_review['created_at']}' and it has a total of {my_review['total_articles']} articles.")


Here is the first review in the list of owned reviews: 'Review test py client' created on '2023-09-06T13:08:40.060Z' and it has a total of 10 articles.


### Listing the first 15 articles sorted by title

In [6]:
review_id = my_review['rayyan_id']
result_params = {
    "start": 0,
    "length": 15,
    "order[0][column]": 5,
    "order[0][dir]": "asc"
    # ... other query parameters ...
}
review_results = rayyan_review.results(review_id, result_params)
print(f'Returned {len(review_results["data"])} record(s) matching {review_results["recordsFiltered"]} record(s) out of {review_results["recordsTotal"]} in total')


Returned 10 record(s) matching 10 record(s) out of 10 in total


### Printing article titles and authors

In [7]:
for index, result in enumerate(review_results['data']):
    print(f'{index+1}: {result["title"]}')
    print(f'    Authors: {", ".join(result["authors"])}')

1: Assessment and characterisation of post-COVID-19 manifestations.
    Authors: Kamal M, Abo Omirah M, Hussein A, Saeed H
2: COVID-19 and arrhythmia: An overview.
    Authors: Varney JA, Dong VS, Tsao T, Sabir MS, Rivera AT, Ghula S, Moriles KE, Cherukuri ML, Fazal R, Azevedo CB, Mohamed RM, Jackson GR, Fleming SE, Rochez DE, Abbas KS, Shah JH, Minh LHN, Osman F, Rafla SM, Huy NT
3: COVID-19 and the liver: overview.
    Authors: Amin M
4: COVID-19 ICU and mechanical ventilation patient characteristics and outcomes-A systematic review and meta-analysis.
    Authors: Chang R, Elhusseiny KM, Yeh YC, Sun WZ
5: COVID-19 in Egypt: Through crisis to adaptation; a gastroenterologist's perspective.
    Authors: El Kassas M, Abdelkader H, Medhat MA
6: COVID-19 in Egypt: Uncovered figures or a different situation?
    Authors: Medhat MA, El Kassas M
7: Diabetes Mellitus and COVID-19: Review Article.
    Authors: Nassar M, Daoud A, Nso N, Medina L, Ghernautan V, Bhangoo H, Nyein A, Mohamed M, Alq

### Including the first article

In [8]:
my_article = review_results['data'][0]
article_id = my_article['id']
customization_plan = {
    "included": 1
}
rayyan_review.customize(review_id, article_id, customization_plan)
print(f'Included article with title: {my_article["title"]}')

Included article with title: Assessment and characterisation of post-COVID-19 manifestations.


### Filtering by included articles

In [9]:
result_params = {
    "start": 0,
    "length": 20,
    "extra[mode]": "included"
}
included_results = rayyan_review.results(review_id, result_params)
print(f'Returned {len(included_results["data"])} record(s) matching {included_results["recordsFiltered"]} record(s) out of {included_results["recordsTotal"]} in total:')
for index, result in enumerate(included_results['data']):
    print(f'{index+1}: {result["title"]}')
    print(f'    Authors: {", ".join(result["authors"])}')


Returned 1 record(s) matching 1 record(s) out of 10 in total:
1: Assessment and characterisation of post-COVID-19 manifestations.
    Authors: Kamal M, Abo Omirah M, Hussein A, Saeed H


### Excluding the next 5 articles

In [10]:
my_articles = review_results['data'][1:6]
customization_plan = {
    "included": -1
}
for article in my_articles:
    article_id = article['id']
    print(f'Excluding article {article_id}...')
    rayyan_review.customize(review_id, article_id, customization_plan)
print(f'Excluded {len(my_articles)} articles from review {review_id}.')

Excluding article 771366...
Excluding article 771365...
Excluding article 771372...
Excluding article 771371...
Excluding article 771367...
Excluded 5 articles from review 2755.


### Excluding the next 3 articles in bluk

In [11]:
my_articles = review_results['data'][7:10]
rayyan_review.bulk_customizations(review_id, "included", -1, (",").join(map(str,[a['id'] for a in my_articles])))

{'missed_etags': {'inclusions_etag': '2023-09-11 20:37:26 UTC',
  'labels_etag': '2023-09-07 13:21:32 UTC'},
 'new_etags': {'inclusions_etag': '2023-09-11 20:37:27 UTC',
  'labels_etag': '2023-09-07 13:21:32 UTC'},
 'customization_ids': [448, 449, 450]}

### Getting the inclusion counts facet for the review

In [12]:
facet_params = {
    "facets[inclusion_counts]": "1"
}
facet_info = rayyan_review.facets(review_id, facet_params)
pprint(facet_info)

{'inclusion_counts': {'collection': [], 'has_evenmore': False, 'more': []}}


### Getting Customizations for `inclusion_decisions` and `labels` of the First 15 Articles Sorted by Title

In [13]:
customization_params = {
    "types[]": ["inclusion_decisions", "labels"]
}

pprint(rayyan_review.get_customizations(review_id, customization_params))

[{'_id': 265,
  'article_id': 776311,
  'created_at': '2023-09-06T13:14:04.274Z',
  'key': 'included',
  'user_id': 266,
  'value': '0'},
 {'_id': 266,
  'article_id': 776311,
  'created_at': '2023-09-06T13:15:27.271Z',
  'key': 'included',
  'user_id': 266,
  'value': '1'},
 {'_id': 267,
  'article_id': 776311,
  'created_at': '2023-09-06T13:15:55.944Z',
  'key': 'included',
  'user_id': 266,
  'value': '-1'},
 {'_id': 268,
  'article_id': 774908,
  'created_at': '2023-09-06T13:15:56.046Z',
  'key': 'included',
  'user_id': 266,
  'value': '-1'},
 {'_id': 269,
  'article_id': 773192,
  'created_at': '2023-09-06T13:15:56.196Z',
  'key': 'included',
  'user_id': 266,
  'value': '-1'}]


### Invite Collaborator to the review

In [14]:
from rayyan.review_access import ReviewAccess
review_access_instance = ReviewAccess(rayyan)

invite_confirmation = review_access_instance.invite(review_id, 2, ["user3@example.com"], "Collaboration on data analysis.")
pprint(invite_confirmation)


''


### Change Collaborator to a Viewer

In [15]:
invite_confirmation = review_access_instance.update_access(review_id, 3, ["user3@example.com"])
pprint(invite_confirmation)


''


### Revoke Viewer access

In [16]:
# invite_confirmation = review_access_instance.update_access(review_id, 3, ["user3@example.com"])
# pprint(invite_confirmation)
my_review_json = rayyan_review.get(review_id)
viewr_id = my_review_json['viewers'][0]['id']

pprint(review_access_instance.revoke(review_id, viewr_id))


''


### Restore Viewer access

In [17]:
pprint(review_access_instance.restore(review_id, viewr_id))
pprint(viewr_id)

''
594


### Delete Viewer access

In [18]:
pprint(review_access_instance.delete_access(review_id, viewr_id))

''


## Notes

### Create note

In [19]:
from rayyan.notes import Notes

notes_instance = Notes(rayyan)
note_id = notes_instance.create_note(review_id, my_articles[0]['id'], text="This article needs further analysis.")
print(f'Added a note to article: {my_articles[0]["title"]} \nNote id: {note_id}')


Added a note to article: Egypt's COVID-19 Recent Happenings and Perspectives: A Mini-Review. 
Note id: 2643


### Update Note

In [20]:
response = notes_instance.update_note(review_id, my_articles[0]['id'], note_id, text="UPDATED This article needs further analysis.")
print(response)


{'status': 'OK'}


### Delete Note

In [21]:
response = notes_instance.delete_note(review_id, my_articles[0]['id'], note_id)
print(response)


{'status': 'OK'}


## ThirdPartyAuth

### Get ThirdPartyAuth link

In [22]:
from rayyan.third_party_auth import ThirdPartyAuth
from rayyan.conf import MENDELEY

third_party_auth_instance = ThirdPartyAuth(rayyan, MENDELEY)
auth_link = third_party_auth_instance.get_auth_link()
pprint(auth_link)

{'code': 302,
 'data': {'redirect_to': 'https://api.mendeley.com/oauth/authorize?client_id=14062&redirect_uri=https%3A%2F%2Fstaging.rayyan.ai%2Fthird_party_auth%2Fmendeley_callback&response_type=code&scope=all'},
 'message': 'Found'}


## Highlights

### Create Highlight

In [25]:
from rayyan.highlight import Highlight
highlight_instance = Highlight(rayyan)

highlight_data = highlight_instance.create_highlight(review_id, category_id=1, keyword="important term")

### Delete Highlight

In [26]:
highlight_data = highlight_instance.delete_highlight(review_id, category_id=1, body_id=1)

## Duplicates
### Trigger dedup job for the review

In [ ]:
from rayyan.duplicates import Duplicates

duplicates_instance = Duplicates(rayyan)

duplicates_instance.add_duplicate(review_id)


In [ ]:
duplicates_instance.get_duplicate(review_id,my_articles[0]['id'])
